# VFB_connect Quickstart Guide


In [4]:
!pip install -r ../../../requirements.txt --quiet
!pip install ../../../ --quiet

# Import lib and initialise object
from vfb_connect.cross_server_tools import VfbConnect
vc = VfbConnect()


## VFB connect API overview

The VFB connect API provides programmatic access to the databases underlying [Virtual Fly Brain](http://virtualflybrain.org). 

At the core of Virtual Fly Brain is a set of curated terms for Drosophila neuro-anatomy organised into a queryable classification, including terms for brain regions, e.g. [nodulus](http://virtualflybrain.org/reports/FBbt_00003680) and neurons e.g. [MBON01](http://virtualflybrain.org/reports/FBbt_00100234). These terms are used to annotate and classify individual brain regions and neurons in images and connectomics data. For example the term MBON01 is used to classify individual [neurons from sources including the CATMAID-FAFB and Neuprint-HemiBrain databases](https://v2.virtualflybrain.org/org.geppetto.frontend/geppetto?q=FBbt_00100234,ListAllAvailableImages). VFB stores both [registered 3D images](http://virtualflybrain.org/reports/VFB_00101382) and [connectomics data](https://v2-dev.virtualflybrain.org/org.geppetto.frontend/geppetto?q=VFB_00101382,ref_neuron_neuron_connectivity_query) (where available) for all of these neurons.

A single VfbConnect object wraps connections and canned queries against all open VFB databases. It includes methods for retreiving metadata about anatomy, individual brain regions and neurons including IDs for these that can be used for queries against other databases (e.g. CATMAID & neuprint).  It provides methods for downloading images and connectomics data. It provides access to sophisticated queries for anatomical classes and individual neurons accoring to their classification & properties. 

### Locations for methods under a `VfbConnect` object.

1. Under `vc.neo_query_wrapper` are 
   1. A set of methods that take lists of IDs as a primary argument and return metadata. 
   2. A set of methods for mapping between VFB IDs and external IDs
2. Directly under `vc` are:
    1. A set of methods that take the names of classes in VFB e.g. 'nodulus' or 'Kenyon cell', or simple query expressions using the names of classes and return metadata about the classes.
    2. A set methods for querying connectivity and similarity
3. Direct access to API queries is provided under the 'nc' and 'oc' attributes for Neo4J and OWL queries respectively. We will not cover details of how to use these here.

Note: available methods and their documentation are easy to explore in DeepNote. Tab completion and type adhead can be used to help find methods. Float your cursor over a method to see its signature and docstring. 

### 1. `vc.neo_query_wrapper` methods overview

**1.1** `vc.neo_query_wrapper` TermInfo queries return the results of a VFB Term Information window as JSON, following the [VFB_JSON standard](https://virtualflybrain.github.io/VFB_json_schema/doc/schema_doc.html), or a summary that can easily be converted into a DataFrame.

In [5]:
# A query for full TermInfo.  This probably produces more information than you will need for most purposes.
vc.neo_query_wrapper.get_type_TermInfo(['FBbt_00003686'])[0].keys()

dict_keys(['label', 'symbol', 'id', 'tags', 'data_source', 'accession', 'parents_label', 'parents_id'])

In [6]:
# A query for summary info
import pandas as pd

summary = vc.neo_query_wrapper.get_type_TermInfo(['FBbt_00003686'])
summary

[{'label': 'KC',
  'symbol': 'KC',
  'id': 'FBbt_00003686',
  'tags': 'Entity|Anatomy|Cell|Class|Nervous_system|Neuron|has_subClass|hasScRNAseq',
  'data_source': '',
  'accession': '',
  'parents_label': 'anterior ectoderm derivative|mushroom body intrinsic neuron',
  'parents_id': 'FBbt_00025991|FBbt_00007484'}]

In [7]:
# A different method is needed to get info about individual neurons

summary = vc.neo_query_wrapper.get_anatomical_individual_TermInfo(['VFB_jrchjrch'])
summary


[{'label': '5-HTPLP01_R',
  'symbol': '5-HTPLP01_R',
  'id': 'VFB_jrchjrch',
  'tags': 'Entity|Adult|Anatomy|Cell|Glutamatergic|Individual|Nervous_system|Neuron|Serotonergic|has_image|has_neuron_connectivity|has_region_connectivity|FlyEM_HB|NBLAST|neuronbridge',
  'data_source': 'neuprint_JRC_Hemibrain_1point1',
  'accession': '1324365879',
  'parents_label': 'adult glutamatergic neuron|adult serotonergic PLP neuron',
  'parents_id': 'FBbt_00058208|FBbt_00110945',
  'xrefs': ':1324365879|:1324365879',
  'templates': 'JRC2018Unisex|JRC_FlyEM_Hemibrain',
  'dataset': 'Xu2020NeuronsV1point1',
  'license': 'https://creativecommons.org/licenses/by/4.0/legalcode'}]

**1.2** The `neo_query_wrapper` also includes methods for mapping between IDs from different sources. 

In [8]:
# Some bodyIDs of HemiBrain neurons from the neuprint DataBase:
bodyIDs = [1068958652, 571424748, 1141631198]
vc.neo_query_wrapper.xref_2_vfb_id(map(str, bodyIDs)) # Note IDs must be strings

{'1068958652': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch06r9'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjwda'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch06r9'},
  {'db': 'neuprint_JRC_Hemibrain_1point1', 'vfb_id': 'VFB_jrchjwda'}],
 '571424748': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch06r6'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjwct'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch06r6'},
  {'db': 'neuprint_JRC_Hemibrain_1point1', 'vfb_id': 'VFB_jrchjwct'}],
 '1141631198': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch05uz'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjw8r'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch05uz'},
  {'db': 'neuprint_JRC_Hemibrain_1point1', 'vfb_id': 'VFB_jrchjw8r'}]}

In [9]:
# xref queries can be constrained by DB. Results can optionally be reversed

vc.neo_query_wrapper.xref_2_vfb_id(map(str, bodyIDs), db = 'neuprint_JRC_Hemibrain_1point1' , reverse_return=True)

{'VFB_jrchjwct': [{'acc': '571424748',
   'db': 'neuprint_JRC_Hemibrain_1point1'}],
 'VFB_jrchjw8r': [{'acc': '1141631198',
   'db': 'neuprint_JRC_Hemibrain_1point1'}],
 'VFB_jrchjwda': [{'acc': '1068958652',
   'db': 'neuprint_JRC_Hemibrain_1point1'}]}

### 2. `vc` direct methods overview

**2.1** Methods that take the names of classes in VFB e.g. 'nodulus' or 'Kenyon cell', or simple query expressions using the names of classes and return metadata about the classes or individual neurons

In [10]:
KC_types = vc.get_subclasses("Kenyon cell", summary=True)
KC_types[0:5]

Running query: FBbt:00003686
Query URL: http://owl.virtualflybrain.org/kbs/vfb/subclasses?object=FBbt%3A00003686&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 37


,label,symbol,id,tags,data_source,accession,parents_label,parents_id
0,Kenyon cell of main calyx,,FBbt_00047926,Entity|Adult|Anatomy|Cell|Cholinergic|Class|Ne...,,,adult Kenyon cell,FBbt_00049825
1,KCab-c,KCab-c,FBbt_00110929,Entity|Adult|Anatomy|Cell|Cholinergic|Class|Ne...,,,alpha/beta surface/core Kenyon cell,FBbt_00049838
2,KCg-m,KCg-m,FBbt_00111061,Entity|Adult|Anatomy|Cell|Cholinergic|Class|Ne...,,,Kenyon cell of main calyx|adult gamma Kenyon cell,FBbt_00047926|FBbt_00049828
3,alpha/beta inner-core Kenyon cell,,FBbt_00049111,Entity|Adult|Anatomy|Cell|Cholinergic|Class|Ne...,,,alpha/beta core Kenyon cell,FBbt_00110929
4,gamma Kenyon cell,,FBbt_00100247,Entity|Anatomy|Cell|Class|Nervous_system|Neuro...,,,Kenyon cell,FBbt_00003686


**2.2** Methods for querying connectivity

Please see [Connectomics Notebook](connectomics.html) for examples